In [72]:
import xarray as xr
import pandas as pd
from pathlib import Path
import glob
import numpy as np
import subprocess
import os

In [148]:
data_path='/scratch/vishald/ug1_monsoonlab/pratham/Datasets/Kerala_August_2019/IMDAA'

In [151]:
precip_files

[]

In [152]:
precip_files=[]
tmp_files=[]

In [153]:
for file in os.listdir(data_path):
    if file.endswith('.nc'):
        if 'APCP' in file:
            precip_files.append(os.path.join(data_path,file))
        elif 'TMP-2m' in file:
            tmp_files.append(os.path.join(data_path,file))

In [154]:
# Open datasets separately
apcp_datasets = [xr.open_dataset(f) for f in precip_files]
tmp_datasets = [xr.open_dataset(f) for f in tmp_files]

# Concatenate along time
apcp_ds = xr.concat(apcp_datasets, dim='time')
tmp_ds = xr.concat(tmp_datasets, dim='time')

In [155]:
apcp_ds.time.size

408

In [156]:
apcp_ds = apcp_ds.sortby('time')
precip_resampled = apcp_ds.resample(time='12h', closed='right', label='right').sum(dim='time')

# Merge both datasets
merged_ds = xr.merge([precip_resampled, tmp_ds], join='inner')

In [157]:
merged_ds.time.size

33

In [158]:
np.allclose(merged_ds.isel(time=0).APCP_sfc,apcp_ds.isel(time=slice(0,12)).APCP_sfc.sum(dim='time'))

True

In [159]:
start_date = str(merged_ds.time.values[0])[:10]
end_date = str(merged_ds.time.values[-1])[:10]

# Save merged dataset with start and end date in filename
output_filename = f'IMDAA0p12_merged_{start_date}_to_{end_date}.nc'
output_path = os.path.join(data_path, output_filename)
merged_ds.to_netcdf(output_path)    

## Regridding to 1deg

In [160]:
deg1_output_path=output_path.replace("0p12","1deg")

In [161]:
try:
    # Build the CDO command
    cdo_command = [
        'cdo', 'remapcon,grid.txt', 
        output_path, 
        deg1_output_path
    ]
    
    # Run the command
    subprocess.run(cdo_command, check=True)
    
    print(f"Regridded 1 degree file saved at {deg1_output_path}")
except e:
    print("Could not create regridded file due to ",e)

cdo    remapcon: YAC first order conservative weights from lonlat (751x501) to lonlat (360x181) grid
cdo    remapcon: YAC first order conservative weights from lonlat (751x501) to lonlat (360x181) grid, with source mask (366720)
Regridded 1 degree file saved at /scratch/vishald/ug1_monsoonlab/pratham/Datasets/Kerala_August_2019/IMDAA/IMDAA1deg_merged_2019-07-24_to_2019-08-09.nc


In [162]:
!du -sh {deg1_output_path}

15M	/scratch/vishald/ug1_monsoonlab/pratham/Datasets/Kerala_August_2019/IMDAA/IMDAA1deg_merged_2019-07-24_to_2019-08-09.nc
